In [118]:
#import keras
import numpy as np
import tensorflow as tf
import glob
from pydub import AudioSegment
from audioset import vggish_keras_embeddings
from time import time

start = time()

list_of_embeddings = []
audio_embedder = vggish_keras_embeddings.VGGishEmbedder(None)

for idx, val in enumerate(sound_slice_generator('I Have a Dream speech by Martin Luther King .Jr HD (subtitled) (Remastered)-vP4iY1TtS3s.webm', 500)):
    waveform, sample_rate = val[0], val[1]
    if val[0].shape[0] < 48000:
        waveform = np.pad(val[0], (0, 48000 - val[0].shape[0]), mode='constant')
    processed_embedding =  audio_embedder.convert_waveform_to_embedding(waveform, sample_rate)

    embedding_final = np.expand_dims(processed_embedding, axis=0)
    list_of_embeddings.append((idx, embedding_final))

predictions = []

model = tf.keras.models.load_model('Models/cheer_applause_LSTM_ThreeLayer_100Epochs.h5')

for idx, v in list_of_embeddings:
    predictions.append((idx, model.predict(v)))
    
end = time()
print(end - start)

W0327 02:34:44.415287 140583340259136 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fd6e077dfd0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0327 02:34:44.651487 140583340259136 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fd72886c048>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0327 02:34:44.839203 140583340259136 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fd7215b56d8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


53.567009687423706


In [111]:
predictions = []

model = tf.keras.models.load_model('Models/cheer_applause_LSTM_ThreeLayer_100Epochs.h5')

for idx, v in list_of_embeddings:
    predictions.append((idx, model.predict(v)))

W0327 02:25:31.327798 140583340259136 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fd7289c1be0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0327 02:25:31.464505 140583340259136 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fd7298566d8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0327 02:25:31.599576 140583340259136 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fd729d4f710>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


In [120]:
for i in predictions:
    if i[1] > 0.55:
        m, s = divmod(i[0]/2, 60)
        print(int(m), 'min', s, 'sec', i[1][0])

0 min 5.0 sec [0.58329225]
0 min 5.5 sec [0.5561318]
0 min 6.0 sec [0.58471155]
0 min 6.5 sec [0.58165264]
0 min 7.0 sec [0.6182258]
0 min 7.5 sec [0.6142911]
0 min 8.0 sec [0.59110683]
0 min 8.5 sec [0.57307845]
0 min 9.0 sec [0.60045505]
0 min 9.5 sec [0.6142905]
0 min 10.0 sec [0.598779]
0 min 10.5 sec [0.6018663]
0 min 11.0 sec [0.59246993]
0 min 11.5 sec [0.597308]
0 min 12.0 sec [0.5777646]
0 min 29.0 sec [0.559415]
0 min 29.5 sec [0.5732046]
0 min 30.0 sec [0.597487]
0 min 30.5 sec [0.5705388]
0 min 31.0 sec [0.606736]
0 min 31.5 sec [0.6055461]
0 min 32.0 sec [0.60121024]
0 min 32.5 sec [0.6008981]
0 min 33.0 sec [0.56967354]
0 min 33.5 sec [0.55883926]
0 min 34.0 sec [0.56926244]
0 min 34.5 sec [0.58118135]
0 min 35.0 sec [0.5807075]
0 min 35.5 sec [0.55646366]
0 min 36.0 sec [0.56189275]
0 min 36.5 sec [0.5534228]
2 min 13.0 sec [0.6124018]
2 min 13.5 sec [0.60939705]
2 min 14.0 sec [0.56948537]
2 min 14.5 sec [0.58655155]
2 min 15.0 sec [0.56715804]
2 min 15.5 sec [0.5579984

In [60]:
import pafy

In [121]:
#url = 'https://www.youtube.com/watch?v=IxAKFlpdcfc' # applause
#url = 'https://www.youtube.com/watch?v=EBGb40yh4SY' # denzel speech
#url = 'https://www.youtube.com/watch?v=408o4ImaHB8' # tennis match
#url = 'https://www.youtube.com/watch?v=vP4iY1TtS3s' # I have a dream speech
url = 'https://www.youtube.com/watch?v=OFPwDe22CoY' # obama speech

In [122]:
video = pafy.new(url)

In [123]:
audiostreams = video.audiostreams
for a in audiostreams:
    print(a.bitrate, a.extension, str(a.get_filesize()/(1024**2)))

50k webm 2.3237075805664062
70k webm 2.941255569458008
128k webm 4.132859230041504
128k m4a 5.646934509277344
160k webm 5.846334457397461


In [124]:
bestaudio = video.getbestaudio()
bestaudio.bitrate

'160k'

In [125]:
file = bestaudio.download()

In [5]:
from pydub import AudioSegment

In [6]:
song = AudioSegment.from_file('Applause.webm')

In [12]:
song.export('Applause.wav', format='wav')

<_io.BufferedRandom name='Applause.wav'>

In [7]:
song = AudioSegment.from_wav('Applause.wav')

In [8]:
len(song)

8009

In [10]:
song.frame_rate

48000

In [17]:
song_slices = song[::500]

In [36]:
from scipy.io import wavfile

sr, wav_data = wavfile.read('Applause.wav')
wav_data.shape

(384449, 2)

In [40]:
import array
import numpy as np
from pydub import AudioSegment

sound = AudioSegment.from_file('Applause.wav')
samples = sound.get_array_of_samples()

In [45]:
np.array(samples).reshape((-1,2)).shape

(384449, 2)

In [39]:
sr, wav_data = wavfile.read('Applause.wav')
wav_data[0]

array([0, 0], dtype=int16)

In [47]:
wav_data.shape

(384449, 2)

In [46]:
sr

48000

In [48]:
sound.frame_rate == sr

True